In [ ]:
import tarfile
import pandas as pd
import numpy as np
from zipfile import ZipFile
from typing import List

# modeling
from keras.utils import pad_sequences
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential
import keras

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
def extract_csv_files(archive_path: str, headers: List[str]) -> pd.DataFrame:
    if archive_path.endswith(".tar.gz"):
        with tarfile.open(archive_path, "r:gz") as tar:
            output = pd.concat([pd.read_csv(tar.extractfile(file), encoding="latin-1")
                               for file in tar.getnames()])

    if archive_path.endswith(".zip"):
        zip_file = ZipFile(archive_path)

        output = pd.concat([pd.read_csv(zip_file.open(csv_file), header=None)
                            for csv_file in zip_file.namelist()])

    if headers:
        output.columns = headers

    return output

### Traffic Preprocessing

In [ ]:
traffic_file_archives = ["../data/citypulse_traffic_raw_data_aarhus_aug_sep_2014.tar.gz",
                         "../data/citypulse_traffic_raw_data_aarhus_oct_nov_2014.zip",
                         ]

headers = ["status", "avg_measured_time", "avg_speed",	"ext_id",
           "median_measured_time", "timestamp", "vehicle_count", "_id", "report_id"]

traffic_data = pd.concat([extract_csv_files(archive, headers)
                          for archive in traffic_file_archives])

traffic_meta_data = pd.read_csv("../data/trafficMetaData.csv")

traffic_data = traffic_data.merge(traffic_meta_data,
                                  how="left",
                                  left_on="report_id",
                                  right_on="REPORT_ID")

traffic_data["timestamp"] = pd.to_datetime(traffic_data["timestamp"])

### Reshape data for LSTM input

In [ ]:
timestamp_index = pd.DataFrame({"timestamp": pd.date_range(pd.to_datetime(
    traffic_data["timestamp"].min()), pd.to_datetime(traffic_data["timestamp"].max()), freq="5min").to_list()})

ext_ids = np.sort(traffic_data["ext_id"].unique())

ts_df = pd.pivot_table(traffic_data[["timestamp", "ext_id", "avg_speed"]],
                       columns="ext_id",
                       index="timestamp",
                       values="avg_speed",
                       dropna=False)

ts_df = timestamp_index.merge(ts_df, how="left", left_on="timestamp",
                              right_index=True).set_index("timestamp", drop=True)

ts_df.interpolate(inplace=True)
ts_df = ts_df.dropna()

ts_df.head()

In [ ]:
ts_df.shape
ts_df.to_csv("../data/traffic_pre_lstm.csv")

In [ ]:
training_sample = ts_df[100:20000]
validation_sample = ts_df[20001:]

In [ ]:
# generate sequences of training data

# set predictive horizon and sequence length
ph = 5
seq_length = 12

# features to randomly sample without replacement
# must be a value between 1 to 449, inclusive 
features = 449 

sensor = pd.Series(training_sample.columns).sample(
    features, replace=False).sort_values().to_list()

print(sensor)

seq_arrays = []
seq_labs = []

training_features = training_sample[sensor]

for i in range(training_features.shape[0]-seq_length-ph):
    seq_arrays.append(training_features.iloc[i:seq_length+i].to_numpy())
    seq_labs.append(training_features.iloc[seq_length+ph+i])

seq_arrays = np.array(seq_arrays, dtype=object).astype(np.float32)
seq_labs = np.array(seq_labs, dtype=object).astype(np.float32)

In [ ]:
# create validation dataset
val_arrays = []
val_labs = []

validation_features = validation_sample[sensor]

for i in range(validation_features.shape[0]-seq_length-ph):
    if i < 12:
        val_arrays.append(validation_features.iloc[:(i+1)].to_numpy())
        val_labs.append(validation_features.iloc[:(i+ph+1)].to_numpy()[-1])
    else:
        val_arrays.append(validation_features.iloc[i:seq_length+i].to_numpy())
        val_labs.append(validation_features.iloc[seq_length+i+ph])

val_arrays = pad_sequences(val_arrays, padding='pre',
                           dtype=object).astype(np.float32)

val_labs = np.array(val_labs, dtype=object).astype(np.float32)

In [ ]:
# define path to save model
model_path = 'lstm_traffic_only_1.keras'

# build the network
features = len(sensor)
output_size = features

model = Sequential()

model.add(LSTM(
    input_shape=(seq_length, features),
    units=64,
    activation="relu",
    return_sequences=True))
model.add(Dropout(0.025))

model.add(LSTM(
    units=32,
    activation="relu",
    return_sequences=True))

model.add(LSTM(
          units=16,
          activation="relu",
          return_sequences=False))

model.add(Dense(units=output_size, activation="relu"))
model.add(Dense(units=output_size, activation="relu"))
model.add(Dense(units=output_size, activation="relu"))

model.add(Dense(units=output_size, activation="linear"))

optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse'])

print(model.summary())

# fit the network
history = model.fit(seq_arrays,
                    seq_labs,
                    epochs=1000,
                    batch_size=500,
                    validation_split=0.05,
                    verbose=2,
                    callbacks=[
                        keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            min_delta=0.1,
                            patience=5,
                            verbose=0,
                            mode='min'),
                        keras.callbacks.ModelCheckpoint(
                            model_path,
                            monitor='val_loss',
                            save_best_only=True,
                            mode='min',
                            verbose=0)
                    ])

# list all data in history
print(history.history.keys())

In [ ]:
# summarize history for Loss/MSE
fig_acc = plt.figure(figsize=(10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss/MSE')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
scores_test = model.evaluate(val_arrays, val_labs, verbose=2)
print('\nMSE: {}'.format(scores_test[1]))

In [ ]:
y_pred_test = model.predict(val_arrays)
y_true_test = val_labs

# aggregating for easier visualization
y_pred_dv = [row.sum() for row in y_pred_test]
y_true_dv = [row.sum() for row in y_true_test]

start = 5000
ts = 1000

fig_verify = plt.figure(figsize=(10, 5))
plt.plot(y_pred_dv[start:start+ts], label='Predicted Value')
plt.plot(y_true_dv[start:start+ts], label='Actual Value')
plt.title('Sum of Average Speed Across Network',
          fontsize=22, fontweight='bold')
plt.ylabel('value')
plt.xlabel('row')
plt.legend()
plt.show()